In [10]:
import sys
import argparse
import os
import json
import re
import spacy
import html
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.add_pipe('sentencizer')

def preproc(text):
    
    modComm = text
    #replace newlines with spaces
    modComm = re.sub(r"\s+", " ", modComm)

    modComm = html.unescape(modComm)

    modComm = re.sub(r"(http|www)\S+", "", modComm)
        
    modComm = ' '.join(modComm.split())

    # get Spacy document for modComm
    # use Spacy document for modComm to create a string.
    doc = nlp(modComm)
    modComm = ""
    for sentence in doc.sents:
        for token in sentence:
            # Replace token with token.lemma
            if token.text.startswith("-") and (not token.lemma_.startswith("-")):
                content = token.lemma_
            else:
                content = token.text
            
            # Uppercase distinction
            if token.text.isupper():
                modComm += content.upper()
            else:
                modComm += content.lower()

            # Write "/POS" after each token.
            # Add the part-of-speech (pos) tag after the token text
            # modComm += "/" + token.tag_
            # Add a space after the token
            # Split tokens with spaces.
            modComm += " "
        # Insert "\n" between sentences.
        # Add a newline character after the sentence
        modComm += "\n"
    return modComm
    # print(token.text, token.pos_, token.dep_)   
    # print(modComm)

directory = '/Users/akilhuang/Desktop/SUDS/NELA2021/NELA2021_0'
files = os.listdir(directory)
index = 0
limit = len(files)
# limit = 5
output_data = []
output_filename = 'output.json'  # Specify the output file name

with open(output_filename, 'w') as output_file:
    while index < limit:
        filename = files[index]
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename)) as f:
                # print(f.read())
                output = dict()
                output['id'] = index
                output["body"] = preproc(f.read())
                output_data.append(output)


        index += 1
with open(output_filename, 'w') as output_file:
    json.dump(output_data, output_file)

In [12]:
import numpy as np
import argparse
import json

import re
import string
import csv

mfd_dict = {}
mfd_lst = [0, 44, 45, 46, 47,48,49,50,51,52,53]

with open('../emfd_amp.csv', 'r') as file:
        reader = csv.reader(file, delimiter=',', quotechar='|')
        headers = next(reader, None)
        # print(headers)
        for row in reader:
            row_dict = {headers[i]: row[i] for i in mfd_lst}
            mfd_dict[row_dict[headers[0]]] = row_dict


file_path = '/Users/akilhuang/Desktop/SUDS/output.json'

# Open the JSON file
with open(file_path, 'r') as file:
    # Load the JSON data
    json_data = json.load(file)

mfd_score = {}

for i in range(len(json_data)):
    care_virtue = 0.0
    fairness_virtue = 0.0
    loyalty_virtue = 0.0
    authority_virtue = 0.0
    sanctity_virtue = 0.0
    
    care_vice = 0
    fairness_vice = 0
    loyalty_vice = 0
    authority_vice = 0
    sanctity_vice = 0
    # print(json_data[i])
    # print(json_data[i]['body'])
    word_length = 0
    
    for token in json_data[i]['body'].split():
        if token in mfd_dict:
            token_c = mfd_dict[token]['care.virtue']
            token_f = mfd_dict[token]['fairness.virtue']
            token_l = mfd_dict[token]['loyalty.virtue']
            token_a = mfd_dict[token]['authority.virtue']
            token_s = mfd_dict[token]['sanctity.virtue']

            decimal_pattern = re.compile(r'\d+\.\d+')
            c = decimal_pattern.search(token_c)
            f = decimal_pattern.search(token_f)
            l = decimal_pattern.search(token_l)
            a = decimal_pattern.search(token_a)
            s = decimal_pattern.search(token_s)

            if c:
                care_virtue += float(c.group())
            if f:
                fairness_virtue += float(f.group())  
            if l:
                loyalty_virtue += float(l.group())  
            if a:
                authority_virtue += float(a.group())  
            if s:
                sanctity_virtue += float(s.group())
            
            token_c = mfd_dict[token]['care.vice']
            token_f = mfd_dict[token]['fairness.vice']
            token_l = mfd_dict[token]['loyalty.vice']
            token_a = mfd_dict[token]['authority.vice']
            token_s = mfd_dict[token]['sanctity.vice']

            decimal_pattern = re.compile(r'\d+\.\d+')
            c = decimal_pattern.search(token_c)
            f = decimal_pattern.search(token_f)
            l = decimal_pattern.search(token_l)
            a = decimal_pattern.search(token_a)
            s = decimal_pattern.search(token_s)

            if c:
                care_vice += float(c.group())
            if f:
                fairness_vice += float(f.group())  
            if l:
                loyalty_vice += float(l.group())  
            if a:
                authority_vice += float(a.group())  
            if s:
                sanctity_vice += float(s.group())
            word_length+=1
    
    care_virtue = care_virtue/word_length
    fairness_virtue = fairness_virtue/word_length
    loyalty_virtue = loyalty_virtue/word_length
    authority_virtue = authority_virtue/word_length
    sanctity_virtue = sanctity_virtue/word_length

    care_vice = care_vice/word_length
    fairness_vice = fairness_vice/word_length
    loyalty_vice = loyalty_vice/word_length
    authority_vice = authority_vice/word_length
    sanctity_vice = sanctity_vice/word_length

    mfd_score[i] = [care_virtue,fairness_virtue,loyalty_virtue, 
                    authority_virtue,sanctity_virtue, care_vice,
                    fairness_vice, loyalty_vice, authority_vice,
                    sanctity_vice]
mfd_score

ZeroDivisionError: float division by zero